In [1]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm

In [2]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)

In [3]:
def parseDataFromFile(fname):
    for l in open(fname):
        yield eval(l)

In [4]:
print "Reading data..."
# data1 = list(parseDataFromFile("/Users/Vernonld/Desktop/book_images_5000.json"))
data2 = list(parseDataFromFile("/Users/Vernonld/Desktop/beer_50000.json"))
print "done"

Reading data...
done


# Question 5
For the reason that question 8 uses the basic model of Q5, so I move the program and answer of Q5 below.

# Question 6

Considering the ‘American IPA’ style, can you come up with a more accurate predictor (e.g. using features from the text, or otherwise)? Write down the feature vector you design, and report its train/test accuracy (1 mark).

I added the feature from the text like 'caramel' or 'amber', both are the typical feature of American IPA. Therefore I think these two features could be ideal


In [5]:
###Question 6 & Question7
def feature(datum):
    feat = [1, datum['beer/ABV'], datum['review/taste'], datum['beer/brewerId']]
    if 'caramel' in datum['review/text'] and 'amber' in datum['review/text'] and 'grapefruit' in datum['review/text']:
        feat.append(1)
    else:
        feat.append(0)
    return feat

X = [feature(b) for b in data2]
y = ["American IPA" in b['beer/style'] for b in data2]

X_train = X[:25000]
y_train = y[:25000]

X_test = X[25000:]
y_test = y[25000:]


In [6]:
# Create a support vector classifier object, with regularization parameter C = 1000
clf = svm.SVC(C=10, kernel = 'linear')  #Penalty parameter C of the error term.
clf.fit(X_train, y_train)
train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)

In [7]:
accuracy_train = sum([t == p for t,p in zip(y_train,train_predictions)]) * 1.0/25000
accuracy_test  = sum([t == p for t,p in zip(y_test,  test_predictions)]) * 1.0/25000
print accuracy_train,accuracy_test

0.9136 0.92188


# Question7
What effect does the regularization constant C have on the training/test performance? Report the train/test accuracy of your predictor from the previous question for C ∈ ⟨0.1, 10, 1000, 100000⟩.

Penalty parameter C of the error term. The C parameter trades off misclassification of training examples against simplicity of the decision surface. A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly by giving the model freedom to select more samples as support vectors.

The lower C required less running time compare to a higher C, and accuracy of the model may increase with the increase of parameter c. But if the feature we choose already make the model accurate enough, then the accuracy would not increase unless we choose new features.

c = 0.1    accuracy_train = 0.9136   accuracy_test = 0.92188

c = 10     accuracy_train = 0.9136   accuracy_test = 0.92188

c = 1000   accuracy_train = 0.9136  accuracy_test = 0.92188

c = 100000 accuracy_train = 0.9136  accuracy_test = 0.92188


# Question5
First, let’s train a predictor that estimates whether a beer is an ‘American IPA’ using two features: [‘beer/ABV’, ‘review/taste’].
Train your predictor using an SVM classifier (see the code provided in class) – remember to train on the first half and test on the second half. Use a regularization constant of C = 1000 as in the code stub. What is the accuracy (percentage of correct classifications) of the predictor on the train and test data? (1 mark)

In [8]:
###TAST 5
def feature(datum):
    feat = [1, datum['beer/ABV'], datum['review/taste']]
    return feat

X = [feature(b) for b in data2]
y = ["American IPA" in b['beer/style'] for b in data2]

X_train = X[:25000]
y_train = y[:25000]

X_test = X[25000:]
y_test = y[25000:]

In [9]:
# Create a support vector classifier object, with regularization parameter C = 1000
clf = svm.SVC(C=1000,kernel = 'linear')
clf.fit(X_train, y_train)

train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)


In [10]:
#Question5 accuracy
accuracy_train = sum([t == p for t,p in zip(y_train,train_predictions)]) * 1.0/25000
accuracy_test  = sum([t == p for t,p in zip(y_test,  test_predictions)]) * 1.0/25000
print accuracy_train,accuracy_test

0.9128 0.92112


# Question 8
Finally, let’s fit a model (for the problem from Q5) using logistic regression. A code stub has been provided to perform logistic regression using the above model on http://jmcauley.ucsd.edu/ cse258/code/homework1.py Code for the log-likelihood has been provided in the code stub (f) but code for the derivative is incomplete (fprime)
Complete the code stub for the derivative (fprime) and provide your solution. What is the log-likelihood of after convergence, and what is the accuracy (on the test set) of the resulting model (1 mark)?

In [11]:
### Question 8
import numpy
import urllib
import scipy.optimize
import random
from math import exp
from math import log


In [12]:
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])
def sigmoid(x):
    return 1.0 / (1 + exp(-x))

In [13]:
# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
    print "ll =", loglikelihood
    return -loglikelihood

In [14]:
# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    for k in range(len(theta)):
        logit_d = 0
        for i in range(len(X)):
        # Fill in code for the derivative
            logit = inner(X[i], theta)
            logit_d += X[i][k] * (1 - sigmoid(logit))
            if not y[i]:
                logit_d -= X[i][k]
        logit_d -= 2 * lam * theta[k]
        dl[k] = logit_d
    # Negate the return value since we're doing gradient *ascent*
    return numpy.array([-x for x in dl])

In [15]:
def feature(datum):
    feat = [1, datum['beer/ABV'], datum['review/taste']]
    return feat

X = [feature(b) for b in data2]
y = ["American IPA" in b['beer/style'] for b in data2]

X_train = X[:len(X)/2]
X_test = X[len(X)/2:]
# If we wanted to split with a validation set:
#X_valid = X[len(X)/2:3*len(X)/4]
#X_test = X[3*len(X)/4:]

In [16]:
# Use a library function to run gradient descent (or you can implement yourself!)
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, y_train, 1.0))
print "Final log likelihood =", -l
print theta
theta = numpy.matrix(theta)
#print "Accuracy = " # Compute the accuracy

ll = -17328.679514
ll = -16341.2013892
ll = -13952.6362653
ll = -6951.99741346
ll = -7358.42557314
ll = -6910.14444961
ll = -6906.4039954
ll = -6892.46364043
ll = -6803.17967996
ll = -6787.73594284
ll = -6783.12927193
ll = -6782.88180995
ll = -6782.29658661
ll = -6780.5430371
ll = -6776.38750021
ll = -6766.5473173
ll = -6747.07517193
ll = -6718.50759263
ll = -6888.22077896
ll = -6716.20422105
ll = -6695.66660154
ll = -6691.5816811
ll = -6690.85883334
ll = -6690.83165269
ll = -6690.83160231
ll = -6690.83159755
Final log likelihood = -6690.83159755
[-1.77315687 -0.4530953   0.67198756]


In [17]:
# Accuracy
def predict(X, theta):
    X = numpy.matrix(X_test)
    y_pred = X_test * theta.T
    for y in y_pred:
        if y < 0:
            yield False
test_predictions = list(predict(X_test, theta))
accuracy_test  = sum([t == p for t,p in zip(y_test,  test_predictions)]) * 1.0/25000
print accuracy_test


0.92188
